In [16]:
# 定义一个简单的循环神经网络（RNN）封装类，旨在为连续和离散输出提供灵活的预测模型

循环神经网络的英文全称是 "Recurrent Neural Network" (RNN)。RNN 是一种强大的神经网络模型，专门设计来处理序列数据，如时间序列数据、自然语言文本、语音等。它的关键特性是网络内部存在循环，使得网络能够在处理序列的每个元素时保持信息的持续流动。

### RNN 原理

1. **循环结构（Recurrent Structure）**：
   RNN 的每个节点（或单元）接收两个输入：当前时间步（Time Step）的输入数据和来自上一个时间步的隐状态（Hidden State）。这种循环结构允许网络维护并更新一个内部状态，该状态编码了到目前为止观察到的序列中的信息。

2. **隐状态（Hidden State）**：
   隐状态是 RNN 的核心，它是网络的内部记忆（Internal Memory），用于存储关于已处理序列的信息。在每个时间步，隐状态会根据当前的输入和上一时间步的隐状态进行更新。

3. **参数共享（Parameter Sharing）**：
   RNN 在处理序列的每个时间步时共享同一组参数（权重和偏置）。这意味着无论输入序列有多长，学习的模型参数数量都保持不变。参数共享提高了模型的数据效率和可扩展性。

4. **时间步循环（Time Step Recurrence）**：
   在处理序列时，RNN 会在每个时间步重复相同的任务，但每次都会更新其隐状态。这种时间步循环允许 RNN 逐步构建对整个序列的理解。

### RNN 的局限性

尽管 RNN 在理论上能够处理任意长度的序列，但在实践中，它们面临一些挑战：

1. **梯度消失和爆炸（Vanishing & Exploding Gradients）**：
   在长序列中，RNN 很难学习长距离依赖关系，这主要是因为在训练过程中梯度可能会急剧减小（消失）或增大（爆炸），导致网络难以学习。

2. **顺序依赖（Sequential Dependency）**：
   由于 RNN 的循环结构，必须按顺序处理序列中的每个元素，这限制了模型训练和推断的并行性。

### RNN 的改进

为了克服标准 RNN 的这些限制，研究人员提出了几种改进的 RNN 结构，如：

- **长短期记忆网络（Long Short-Term Memory, LSTM）**：
  LSTM 通过引入一种复杂的门控机制（Gating Mechanism）来控制信息的流入、流出和遗忘，从而更有效地捕获长距离依赖关系。

- **门控循环单元（Gated Recurrent Unit, GRU）**：
  GRU 是 LSTM 的一个变体，具有更简化的结构，但仍然通过类似的门控机制来管理信息流，提高了处理长序列的能力。

RNN 及其变种在多个领域展现出了强大的性能，使得它们成为处理序列数据时的重要工具。

# 1.) Create a function that takes X, y data and allows you to fit/predict similar to a scikit learn function.

In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 类定义和初始化

# SimpleRNNWrapper类的构造函数初始化模型的基本参数，
# 包括输出类型（连续或离散）、时间序列滞后观察数（lags）、RNN层的单元数、训练的轮数（epochs）和批处理大小（batch_size）。
class SimpleRNNWrapper:
    def __init__(self, output_type='continuous', lags=1, rnn_units=50, epochs=10, batch_size=32):
        """
        Initialize the RNN wrapper.

        Parameters:
        - output_type: 'continuous' for regression tasks, 'discrete' for classification.
        - lags: Number of lag observations to use as input features.
        - rnn_units: Number of units in the RNN layer.
        - epochs: Number of epochs to train the model.
        - batch_size: Batch size for training.
        """
        self.output_type = output_type
        self.lags = lags
        self.rnn_units = rnn_units
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.scaler = MinMaxScaler()
        self.label_encoder = None
        if output_type == 'discrete':
            self.label_encoder = LabelEncoder() 
            # 根据输出类型（连续或离散），可能还会初始化一个标签编码器（LabelEncoder）


# 数据预处理
            
# _preprocess_data方法：负责数据预处理，包括特征缩放（使用MinMaxScaler）和将数据结构化为序列形式，以适应RNN的输入需求       
    def _preprocess_data(self, X, y=None):
        """
        Preprocess the data: scale features, encode labels (if discrete), and structure into sequences.
        """
        X_scaled = self.scaler.fit_transform(X)
        X_seq = np.array([X_scaled[i:i+self.lags] for i in range(len(X_scaled)-self.lags)])

        if y is not None:
            if self.output_type == 'continuous':
                y_seq = y[self.lags:]
            else:  # Discrete
                y_encoded = self.label_encoder.fit_transform(y)
                y_seq = to_categorical(y_encoded[self.lags:])
                # 对于离散输出，还会对目标变量进行编码（使用LabelEncoder）并转换为独热编码（使用to_categorical）
            return X_seq, y_seq

        return X_seq


# 模型构建   
# _build_model方法：根据初始化时指定的参数构建RNN模型。模型由一个简单的RNN层（SimpleRNN）和一个输出层组成。
    def _build_model(self, input_shape, output_shape):
        """
        Build the RNN model based on specified parameters.
        """
        self.model = Sequential()
        self.model.add(SimpleRNN(self.rnn_units, input_shape=input_shape))
        if self.output_type == 'continuous':
            self.model.add(Dense(1))
        else:  # Discrete
            self.model.add(Dense(output_shape, activation='softmax'))
            # 输出层对于连续任务使用一个单元（无激活函数），对于离散任务则使用与目标类别数相同的单元数，并采用softmax激活函数。

        self.model.compile(optimizer='adam', loss='mse' if self.output_type == 'continuous' else 'categorical_crossentropy', metrics=['accuracy'])
        # 损失函数根据任务类型（连续或离散）分别设置为均方误差（mse）或分类交叉熵（categorical_crossentropy）
        
        

# 模型训练
# fit方法：负责模型训练。
# 首先，它会调用_preprocess_data方法对输入数据进行预处理，然后调用_build_model构建模型，并使用预处理后的数据训练模型。
        
    def fit(self, X, y):
        """
        Fit the RNN model to the data.
        """
        X_seq, y_seq = self._preprocess_data(X, y)
        self._build_model(input_shape=(X_seq.shape[1], X_seq.shape[2]), output_shape=y_seq.shape[1])
        self.model.fit(X_seq, y_seq, epochs=self.epochs, batch_size=self.batch_size)


# 模型预测
# predict方法：用于模型预测。
# 它首先调用_preprocess_data方法对输入数据X进行预处理，然后使用训练好的模型进行预测。
# 对于离散输出，预测结果会经过LabelEncoder的逆转换，以还原为原始的标签。     
    def predict(self, X):
        """
        Predict using the trained RNN model.
        """
        X_seq = self._preprocess_data(X)
        predictions = self.model.predict(X_seq)
        if self.output_type == 'discrete':
            return self.label_encoder.inverse_transform(np.argmax(predictions, axis=1))
        return predictions.flatten()



总结：
这个SimpleRNNWrapper类提供了一个封装好的RNN模型，适用于处理序列数据的回归（连续输出）和分类（离散输出）任务。通过这种封装方式，用户可以轻松地在自己的数据集上应用RNN，而无需关心数据预处理和模型构建的复杂细节。此外，这个类还处理了数据的标准化和标签编码，确保数据格式适合神经网络，并在预测时还原了原始的标签，增强了模型的易用性和可解释性。